In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import librosa as lb
import librosa.display as lbd
import os
import pickle
tf.random.set_seed(100)
np.random.seed(100)

In [15]:
train=pd.read_csv('/content/drive/My Drive/MEMS Google Colabs/csv_data/train.csv')
val=pd.read_csv('/content/drive/My Drive/MEMS Google Colabs/csv_data/val.csv')
train.head()

,Unnamed: 0,Start_time,End_Time,Wheezing,Patient_ID,Mode,File_Name,Audio_Name
0,5336,2.1450,7.788,0,200,mc,200_2p3_Al_mc_AKGC417L,200_2p3_Al_mc_AKGC417L_0.wav
1,3058,11.1600,15.401,1,160,mc,160_1b4_Al_mc_AKGC417L,160_1b4_Al_mc_AKGC417L_2.wav
2,1945,10.6500,13.321,0,135,mc,135_2b1_Al_mc_LittC2SE,135_2b1_Al_mc_LittC2SE_5.wav
3,1475,6.9447,7.997,1,124,sc,124_1b1_Ar_sc_Litt3200,124_1b1_Ar_sc_Litt3200_5.wav
4,2938,14.5660,16.101,0,154,mc,154_1b3_Al_mc_AKGC417L,154_1b3_Al_mc_AKGC417L_8.wav


In [16]:
ytrain=train.Wheezing
yval=val.Wheezing
yval

0       0
1       0
2       0
3       0
4       0
       ..
1720    0
1721    0
1722    0
1723    0
1724    0
Name: Wheezing, Length: 1725, dtype: int64

In [17]:
pkl_file = open('/content/drive/My Drive/MEMS Google Colabs/saved_pickles/mfcc_train.pkl', 'rb')
mfcc_train = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/MEMS Google Colabs/saved_pickles/cstft_train.pkl', 'rb')
cstft_train = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/MEMS Google Colabs/saved_pickles/mSpec_train.pkl', 'rb')
mSpec_train = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/MEMS Google Colabs/saved_pickles/mfcc_val.pkl', 'rb')
mfcc_val = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/MEMS Google Colabs/saved_pickles/cstft_val.pkl', 'rb')
cstft_val = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('/content/drive/My Drive/MEMS Google Colabs/saved_pickles/mSpec_val.pkl', 'rb')
mSpec_val = pickle.load(pkl_file)
pkl_file.close()

In [18]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0.00001,mode='min')
]

In [19]:
mfcc_input=keras.layers.Input(shape=(20,259,1),name="mfccInput")
layer_mfcc = {}
layer_mfcc[0]=keras.layers.Conv2D(32,5,strides=(1,3),padding='same')(mfcc_input)
layer_mfcc[1]=keras.layers.BatchNormalization()(layer_mfcc[0])
layer_mfcc[2]=keras.layers.Activation(keras.activations.relu)(layer_mfcc[1])

mfcc_output=keras.layers.GlobalMaxPooling2D()(layer_mfcc[2])

mfcc_model=keras.Model(mfcc_input, mfcc_output, name="mfccModel")

In [21]:
mfcc_model.save('./test.h5', overwrite=True)